<a href="https://colab.research.google.com/github/s528661/nlp100/blob/JDLA_Engeneer_20240503/NLP100Knock_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第8章: ニューラルネット  
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．  
なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

In [50]:
!pip install mecab-python3

In [51]:
import gc
import shutil
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

# import dask.array as da

import lightgbm as lgb

# Neural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import RidgeClassifier
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [52]:
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras import losses

## 70. 単語ベクトルの和による特徴量  

In [53]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate

!unzip ./NewsAggregatorDataset.zip

--2024-05-06 03:58:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip.1’

NewsAggregatorDatas     [     <=>            ]  27.87M   382KB/s    in 2m 10s  

2024-05-06 04:00:27 (220 KB/s) - ‘NewsAggregatorDataset.zip.1’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
replace 2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [54]:
data0 = pd.read_csv('newsCorpora.csv',delimiter='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [55]:
data0

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
...,...,...,...,...,...,...,...,...
422414,422933,Surgeons to remove 4-year-old's rib to rebuild...,http://www.cbs3springfield.com/story/26378648/...,WSHM-TV,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.cbs3springfield.com,1409229190251
422415,422934,Boy to have surgery on esophagus after battery...,http://www.wlwt.com/news/boy-to-have-surgery-o...,WLWT Cincinnati,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wlwt.com,1409229190508
422416,422935,Child who swallowed battery to have reconstruc...,http://www.newsnet5.com/news/local-news/child-...,NewsNet5.com,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.newsnet5.com,1409229190771
422417,422936,Phoenix boy undergoes surgery to repair throat...,http://www.wfsb.com/story/26368078/phoenix-boy...,WFSB,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wfsb.com,1409229191071


In [56]:
data = data0[data0['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [57]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [58]:
data['TITLE'] = data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-58-92b564d62c6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TITLE'] = data['TITLE'].apply(lambda x: preprocessing(x))


In [59]:
vectorizer = TfidfVectorizer(min_df=10, ngram_range=(1, 2)) # 1-gram, 2-gramでTfidfを計算
X = vectorizer.fit_transform(data['TITLE']).toarray()
X = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())

In [60]:
X_train0, X_test = train_test_split(X, test_size=0.2, shuffle=True)

In [61]:
X_train, X_val = train_test_split(X_train0, test_size=0.1, shuffle=True)

In [62]:
y_train = data.iloc[X_train.index]['CATEGORY'].map({'b': 0, 'e': 1, 't': 2, 'm': 3})
y_val = data.iloc[X_val.index]['CATEGORY'].map({'b': 0, 'e': 1, 't': 2, 'm': 3})
y_test = data.iloc[X_test.index]['CATEGORY'].map({'b': 0, 'e': 1, 't': 2, 'm': 3})

In [63]:
if os.path.exists("data"):
  shutil.rmtree("data")
os.mkdir("data")

os.mkdir("data/train")
os.mkdir("data/val")
os.mkdir("data/test")

In [64]:
X_train.to_pickle('data/train/X_train.pkl')
X_val.to_pickle('data/val/X_val.pkl')
X_test.to_pickle('data/test/X_test.pkl')

In [65]:
y_train.to_pickle('data/train/y_train.pkl')
y_val.to_pickle('data/val/y_val.pkl')
y_test.to_pickle('data/test/y_test.pkl')

## 71. 単層ニューラルネットワークによる予測

In [66]:
n_dim = X.shape[1]
n_hidden = 32
n_class = len(set(data['CATEGORY']))

In [67]:
class CustomModel(tf.keras.Model):
  def __init__(self):
    super(CustomModel, self).__init__()
    self.inputl = layers.Dense(activation="relu", name='Input',  units=n_dim)
    self.hiddenl = layers.Dense(activation="relu", name='Hidden',  units=n_hidden)
    self.outputl = layers.Dense(activation="softmax", name='Output', units=n_class)

  def call(self, x, training=None):
    x = self.inputl(x)
    #layers.Dropout(0.5)
    x = self.hiddenl(x)
    x = self.outputl(x)
    return x

In [68]:
model = CustomModel()

In [69]:
X_train = pd.read_pickle('data/train/X_train.pkl')
X_val = pd.read_pickle('data/val/X_val.pkl')
X_test = pd.read_pickle('data/test/X_test.pkl')

In [70]:
y_train = pd.read_pickle('data/train/y_train.pkl')
y_val = pd.read_pickle('data/val/y_val.pkl')
y_test = pd.read_pickle('data/test/y_test.pkl')

In [71]:
loss = losses.CategoricalCrossentropy()

# def categorycal_cross_entropy(y_val, y_pred):
#     # int型だとエラーになりました
#     y_val = tf.cast(y_val, tf.float32)
#     y_pred = tf.cast(y_pred, tf.float32)
#     y_pred = tf.clip_by_value(y_pred, 1e-6, y_pred)  # log(0)回避用
#     # -Σ p * log(q)
#     loss = -tf.reduce_sum(y_val * tf.math.log(y_pred), axis=1)
#     return tf.reduce_mean(loss)

In [72]:
model.compile(optimizer="adam", loss=loss)

In [73]:
y_train_onehot = tf.one_hot(y_train, n_class)

In [75]:
model.fit(X_train.values, y_train_onehot, epochs=10, batch_size=16)

Epoch 1/10
601/601 [==============================] - 145s 238ms/step - loss: 0.4790
Epoch 2/10
601/601 [==============================] - 138s 229ms/step - loss: 0.1710
Epoch 3/10
601/601 [==============================] - 137s 228ms/step - loss: 0.0607
Epoch 4/10
601/601 [==============================] - 136s 227ms/step - loss: 0.0199
Epoch 5/10
601/601 [==============================] - 151s 251ms/step - loss: 0.0125
Epoch 6/10
601/601 [==============================] - 140s 233ms/step - loss: 0.0074
Epoch 7/10
601/601 [==============================] - 142s 236ms/step - loss: 0.0064
Epoch 8/10
601/601 [==============================] - 145s 241ms/step - loss: 0.0056
Epoch 9/10
601/601 [==============================] - 141s 235ms/step - loss: 0.0039
Epoch 10/10
601/601 [==============================] - 146s 243ms/step - loss: 0.0042


In [76]:
model.summary()

Model: "custom_model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (Dense)               multiple                  9458700   
                                                                 
 Hidden (Dense)              multiple                  98432     
                                                                 
 Output (Dense)              multiple                  132       
                                                                 
Total params: 9557264 (36.46 MB)
Trainable params: 9557264 (36.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
